# Capstone Project: Comparing LA vs NYC Neighborhoods

## Import Libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Libraries imported.


In [2]:
pd.options.mode.chained_assignment = None

## Importing and Cleaning the Raw Data to make final Location DataFrame

### Import Zipcodes CSV

In [3]:
US_loc_data = pd.read_csv('zip_code_database.csv', converters={'zip': lambda x: str(x)})

### Cleaning the data and making a table with just New York and Los Angeles Zipcodes and Latitude/Longitude data

In [4]:
CAlocdata = US_loc_data[US_loc_data['state']=='CA']
NYlocdata = US_loc_data[US_loc_data['state']=='NY']

LAlocdata = CAlocdata[(CAlocdata['county']=='Los Angeles County')]
LAlocdata.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
38117,90001,STANDARD,0,Los Angeles,"Firestone Park, Firestone Pk",NaN,CA,Los Angeles County,America/Los_Angeles,323,NaN,US,33.97,-118.24,53610
38118,90002,STANDARD,0,Los Angeles,"August F. Haw, Watts",NaN,CA,Los Angeles County,America/Los_Angeles,"323,562",NaN,US,33.94,-118.24,47820
38119,90003,STANDARD,0,Los Angeles,NaN,Broadway Manchester,CA,Los Angeles County,America/Los_Angeles,323,NaN,US,33.96,-118.27,65150
38120,90004,STANDARD,0,Los Angeles,Oakwood,NaN,CA,Los Angeles County,America/Los_Angeles,323,NaN,US,34.07,-118.31,50840
38121,90005,STANDARD,0,Los Angeles,Sanford,NaN,CA,Los Angeles County,America/Los_Angeles,"213,310,323,562,626,818",NaN,US,34.06,-118.31,29510


In [5]:
LAlocdata.reset_index(inplace=True)

In [6]:
LAlocdata.drop(columns=['index', 'type', 'decommissioned', 'acceptable_cities', 'unacceptable_cities', 'timezone', 'area_codes', 'country', 'world_region',
                       'irs_estimated_population_2015', 'state'], inplace=True)
LAlocdata.head()

,zip,primary_city,county,latitude,longitude
0,90001,Los Angeles,Los Angeles County,33.97,-118.24
1,90002,Los Angeles,Los Angeles County,33.94,-118.24
2,90003,Los Angeles,Los Angeles County,33.96,-118.27
3,90004,Los Angeles,Los Angeles County,34.07,-118.31
4,90005,Los Angeles,Los Angeles County,34.06,-118.31


In [7]:
LAlocdata.rename(columns={'primary_city':'city'}, inplace=True)
LAlocdata.head()

,zip,city,county,latitude,longitude
0,90001,Los Angeles,Los Angeles County,33.97,-118.24
1,90002,Los Angeles,Los Angeles County,33.94,-118.24
2,90003,Los Angeles,Los Angeles County,33.96,-118.27
3,90004,Los Angeles,Los Angeles County,34.07,-118.31
4,90005,Los Angeles,Los Angeles County,34.06,-118.31


In [8]:
NYClocdata = NYlocdata[(NYlocdata['county']=='New York County') | (NYlocdata['county']=='Bronx County') | (NYlocdata['county']=='Kings County') |
                      (NYlocdata['county']=='Queens County') | (NYlocdata['county']=='Richmond County')]
NYClocdata.head()

,zip,type,decommissioned,primary_city,acceptable_cities,unacceptable_cities,state,county,timezone,area_codes,world_region,country,latitude,longitude,irs_estimated_population_2015
3699,10001,STANDARD,0,New York,NaN,"Empire State, G P O, Greeley Square, Macys Fin...",NY,New York County,America/New_York,"718,917,347,646",NaN,US,40.75,-74.00,19710
3700,10002,STANDARD,0,New York,Knickerbocker,"Manhattan, New York City, Ny, Ny City, Nyc",NY,New York County,America/New_York,718,NaN,US,40.71,-73.99,71450
3701,10003,STANDARD,0,New York,NaN,"Cooper, Manhattan",NY,New York County,America/New_York,"212,347,646,718,917",NaN,US,40.73,-73.99,39470
3702,10004,STANDARD,0,New York,Bowling Green,NaN,NY,New York County,America/New_York,"212,347,646,718,917",NaN,US,40.69,-74.02,4380
3703,10005,STANDARD,0,New York,Wall Street,"Manhattan, Nyc",NY,New York County,America/New_York,"347,718,212,646,917",NaN,US,40.71,-74.01,8600


In [9]:
NYClocdata.reset_index(inplace=True)
NYClocdata.drop(columns=['index', 'type', 'decommissioned', 'acceptable_cities', 'unacceptable_cities', 'timezone', 'area_codes', 'country', 'world_region',
                       'irs_estimated_population_2015', 'state'], inplace=True)

In [10]:
NYClocdata.rename(columns={'primary_city':'city'}, inplace=True)

In [11]:
nyladata = pd.concat([NYClocdata, LAlocdata], ignore_index=True)
nyladata.rename(columns={'primary_city':'city'}, inplace=True)

### Final Location Data Table

In [12]:
nyladata

,zip,city,county,latitude,longitude
0,10001,New York,New York County,40.75,-74.00
1,10002,New York,New York County,40.71,-73.99
2,10003,New York,New York County,40.73,-73.99
3,10004,New York,New York County,40.69,-74.02
4,10005,New York,New York County,40.71,-74.01
5,10006,New York,New York County,40.71,-74.01
6,10007,New York,New York County,40.71,-74.01
7,10008,New York,New York County,40.71,-73.99
8,10009,New York,New York County,40.73,-73.98
9,10010,New York,New York County,40.74,-73.98


## Initial maps of NYC and LA

In [13]:
addressNYC = 'New York City, NY'

geolocator = Nominatim(user_agent='Final_project')
locationNYC = geolocator.geocode(addressNYC)
latNYC = locationNYC.latitude
longNYC = locationNYC.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latNYC, longNYC))

The geograpical coordinate of New York City are 40.7308619, -73.9871558.


In [14]:
addressLA = 'Los Angeles, CA'

geolocator = Nominatim(user_agent='Final_project')
locationLA = geolocator.geocode(addressLA)
latLA = locationLA.latitude
longLA = locationLA.longitude
print('The geograpical coordinate of Los Angeles are {}, {}.'.format(latLA, longLA))

The geograpical coordinate of Los Angeles are 34.0536834, -118.2427669.


### NYC map with zipcode popups

In [15]:
map_newyork = folium.Map(location=[latNYC, longNYC], zoom_start=10)

for lat, lng, zipc, city in zip(NYClocdata['latitude'], NYClocdata['longitude'], NYClocdata['zip'], NYClocdata['city']):
    label = '{}, {}'.format(zipc, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

### LA map with zipcode popups

In [16]:
map_losangeles = folium.Map(location=[latLA, longLA], zoom_start=10)

for lat, lng, zipc, city in zip(LAlocdata['latitude'], LAlocdata['longitude'], LAlocdata['zip'], LAlocdata['city']):
    label = '{}, {}'.format(zipc, city)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_losangeles)  
    
map_losangeles

## Bringing in Foursquare data

### Defining Foursquare credentials:

In [17]:
CLIENT_ID = '5PYOPFAT54OJP3NYENDDUIU4RW1DNANTJKABT5TNGBDT2RY2' # your Foursquare ID
CLIENT_SECRET = 'OTARQI2XSFCXBYP2BPKHZCYQAXKQKP1XPG1V2SYAYIISRGNK' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100
radius = 500

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5PYOPFAT54OJP3NYENDDUIU4RW1DNANTJKABT5TNGBDT2RY2
CLIENT_SECRET:OTARQI2XSFCXBYP2BPKHZCYQAXKQKP1XPG1V2SYAYIISRGNK


In [18]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Zip', 
                  'Zip Latitude', 
                  'Zip Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [19]:
nyladata.head()

,zip,city,county,latitude,longitude
0,10001,New York,New York County,40.75,-74.00
1,10002,New York,New York County,40.71,-73.99
2,10003,New York,New York County,40.73,-73.99
3,10004,New York,New York County,40.69,-74.02
4,10005,New York,New York County,40.71,-74.01


In [20]:
nyla_venues = getNearbyVenues(nyladata['zip'], nyladata['latitude'], nyladata['longitude'])

10001
10002
10003
10004
10005
10006
10007
10008
10009
10010
10011
10012
10013
10014
10015
10016
10017
10018
10019
10020
10021
10022
10023
10024
10025
10026
10027
10028
10029
10030
10031
10032
10033
10034
10035
10036
10037
10038
10039
10040
10041
10043
10044
10045
10046
10047
10048
10055
10060
10065
10069
10072
10075
10079
10080
10081
10082
10087
10090
10094
10095
10096
10098
10099
10101
10102
10103
10104
10105
10106
10107
10108
10109
10110
10111
10112
10113
10114
10115
10116
10117
10118
10119
10120
10121
10122
10123
10124
10125
10126
10128
10129
10130
10131
10132
10133
10138
10149
10150
10151
10152
10153
10154
10155
10156
10157
10158
10159
10160
10161
10162
10163
10164
10165
10166
10167
10168
10169
10170
10171
10172
10173
10174
10175
10176
10177
10178
10179
10184
10185
10196
10197
10199
10203
10211
10212
10213
10242
10249
10256
10257
10258
10259
10260
10261
10265
10268
10269
10270
10271
10272
10273
10274
10275
10276
10277
10278
10279
10280
10281
10282
10285
10286
10292
10301
10302
1030

In [21]:
nyladata.shape

(863, 5)

In [23]:
nyla_venues.head()

,Zip,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,10001,40.75,-74.0,Porteño,40.750443,-74.002407,Argentinian Restaurant
1,10001,40.75,-74.0,Bluestone Lane,40.752277,-73.998930,Coffee Shop
2,10001,40.75,-74.0,Jun-Men Ramen Bar,40.747956,-74.000301,Ramen Restaurant
3,10001,40.75,-74.0,Hudson Market,40.750678,-74.002087,Grocery Store
4,10001,40.75,-74.0,Sullivan Street Bakery,40.747333,-74.000587,Bakery


In [29]:
nyla_venues.groupby('Zip').count().head()

,Zip Latitude,Zip Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Zip,,,,,,
10001,100,100,100,100,100,100
10002,25,25,25,25,25,25
10003,100,100,100,100,100,100
10004,29,29,29,29,29,29
10005,100,100,100,100,100,100


## Analyzing the data

### One-hot encoding:

In [31]:
nyla_onehot = pd.get_dummies(nyla_venues[['Venue Category']], prefix="", prefix_sep="")
nyla_onehot['Zip'] = nyla_venues['Zip']
fixed_columns = [nyla_onehot.columns[-1]] + list(nyla_onehot.columns[:-1])
nyla_onehot = nyla_onehot[fixed_columns]

nyla_onehot.head()

,Zip,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Baseball Diamond,College Bookstore,College Cafeteria,College Classroom,College Gym,College Quad,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Costume Shop,Country Dance Club,Courthouse,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flight School,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Li

In [32]:
nyla_onehot.shape

(21978, 497)

In [33]:
nyla_grouped = nyla_onehot.groupby('Zip').mean().reset_index()
nyla_grouped.head()

,Zip,ATM,Accessories Store,Adult Boutique,Advertising Agency,Afghan Restaurant,African Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,Airport Tram,American Restaurant,Animal Shelter,Antique Shop,Aquarium,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auditorium,Australian Restaurant,Auto Dealership,Auto Garage,Auto Workshop,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Beach,Beach Bar,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bike Trail,Bistro,Board Shop,Boat or Ferry,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Center,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Campground,Candy Store,Cantonese Restaurant,Car Wash,Caribbean Restaurant,Casino,Caucasian Restaurant,Cemetery,Check Cashing Service,Cheese Shop,Child Care Service,Chinese Restaurant,Chocolate Shop,Church,Churrascaria,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Administrative Building,College Arts Building,College Baseball Diamond,College Bookstore,College Cafeteria,College Classroom,College Gym,College Quad,College Theater,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Comic Shop,Community Center,Concert Hall,Construction & Landscaping,Convenience Store,Convention Center,Cooking School,Cosmetics Shop,Costume Shop,Country Dance Club,Courthouse,Coworking Space,Creperie,Cruise,Cuban Restaurant,Cultural Center,Cupcake Shop,Currency Exchange,Cycle Studio,Czech Restaurant,Dance Studio,Daycare,Deli / Bodega,Dentist's Office,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dive Shop,Doctor's Office,Dog Run,Donburi Restaurant,Dongbei Restaurant,Donut Shop,Drugstore,Dry Cleaner,Dumpling Restaurant,Duty-free Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Film Studio,Financial or Legal Service,Fish & Chips Shop,Fish Market,Fishing Store,Flea Market,Flight School,Flower Shop,Food,Food & Drink Shop,Food Court,Food Service,Food Stand,Food Truck,Football Stadium,Fountain,Frame Store,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,General Travel,German Restaurant,Gift Shop,Gluten-free Restaurant,Golf Course,Golf Driving Range,Gourmet Shop,Greek Restaurant,Grocery Store,Gun Range,Gym,Gym / Fitness Center,Gym Pool,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Heliport,Herbs & Spices Store,High School,Himalayan Restaurant,Historic Site,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hunan Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Indoor Play Area,Industrial Estate,Insurance Office,Intersection,Irish Pub,Island,Israeli Restaurant,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lawyer,Leather Goods Store,Lebanese Restaurant,Library,Light Rail Station,Li

Confirm shape of nyla_grouped:

In [35]:
nyla_grouped.shape

(817, 497)

Confirm that no rows were lost during onehot encoding and subsequent grouping:

In [39]:
nyla_grouped.shape[0] == nyla_venues.groupby('Zip').count().shape[0]

True

### Dataframe with top 10 venues per zip:

In [40]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [44]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Zip']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
zip_venues_sorted = pd.DataFrame(columns=columns)
zip_venues_sorted['Zip'] = nyla_grouped['Zip']

for ind in np.arange(nyla_grouped.shape[0]):
    zip_venues_sorted.iloc[ind, 1:] = return_most_common_venues(nyla_grouped.iloc[ind, :], num_top_venues)

zip_venues_sorted.head()

,Zip,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,10001,Art Gallery,Coffee Shop,Italian Restaurant,Chinese Restaurant,Pizza Place,Health & Beauty Service,Theater,Pool,Cocktail Bar,Tapas Restaurant
1,10002,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink
2,10003,Coffee Shop,Japanese Restaurant,Cycle Studio,Indie Theater,Grocery Store,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Chinese Restaurant,Soba Restaurant
3,10004,Food Truck,Seafood Restaurant,Bike Rental / Bike Share,Theme Park Ride / Attraction,Pier,Baseball Field,Gift Shop,Park,Gym / Fitness Center,Beer Garden
4,10005,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Park,Pizza Place,Wine Shop,Gym / Fitness Center,Greek Restaurant,Café


## Cluster the Zips:

In [75]:
kclusters = 10
nyla_grouped_clustering = nyla_grouped.drop('Zip', 1)
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(nyla_grouped_clustering)

kmeans.labels_

array([1, 4, 1, 1, 1, 1, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 4,
       1, 1, 1, 4, 4, 1, 1, 1, 1, 1, 4, 4, 1, 4, 4, 1, 4, 1, 1, 1, 4, 4,
       1, 4, 4, 4, 4, 4, 4, 1, 1, 1, 4, 4, 1, 4, 4, 4, 1, 4, 4, 4, 4, 4,
       4, 4, 1, 4, 4, 4, 4, 4, 4, 1, 4, 4, 1, 1, 1, 4, 4, 4, 4, 4, 4, 4,
       1, 4, 4, 1, 4, 1, 1, 1, 1, 1, 1, 1, 1, 4, 4, 1, 4, 4, 1, 4, 1, 1,
       1, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 1, 4, 4, 4, 4,
       4, 4, 1, 1, 1, 4, 5, 4, 4, 4, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 5, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 5, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 5, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1,
       1, 5, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 5, 5, 5, 1, 5, 5, 1, 5, 5, 5,
       1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 5, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1,

In [93]:
nyladata.rename(columns={'zip':'Zip'}, inplace=True)

nyla_merged = pd.merge(nyladata,zip_venues_sorted, how='outer', on='Zip', indicator=True)

nyla_merged = nyla_merged[nyla_merged['_merge']=='both']

drop_list = list(zip_venues_sorted.columns)
drop_list.pop(0)
drop_list.append('_merge')
drop_list

['1st Most Common Venue',
 '2nd Most Common Venue',
 '3rd Most Common Venue',
 '4th Most Common Venue',
 '5th Most Common Venue',
 '6th Most Common Venue',
 '7th Most Common Venue',
 '8th Most Common Venue',
 '9th Most Common Venue',
 '10th Most Common Venue',
 '_merge']

In [94]:
nyla_merged = nyla_merged.drop(columns=drop_list)

In [95]:
nyla_merged['Cluster'] = kmeans.labels_

In [96]:
nyla_merged.head()

,Zip,city,county,latitude,longitude,Cluster
0,10001,New York,New York County,40.75,-74.00,1
1,10002,New York,New York County,40.71,-73.99,4
2,10003,New York,New York County,40.73,-73.99,1
3,10004,New York,New York County,40.69,-74.02,1
4,10005,New York,New York County,40.71,-74.01,1


In [97]:
nyla_merged = pd.merge(nyla_merged,zip_venues_sorted, how='outer', on='Zip', indicator=True)
nyla_merged.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
0,10001,New York,New York County,40.75,-74.00,1,Art Gallery,Coffee Shop,Italian Restaurant,Chinese Restaurant,Pizza Place,Health & Beauty Service,Theater,Pool,Cocktail Bar,Tapas Restaurant,both
1,10002,New York,New York County,40.71,-73.99,4,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink,both
2,10003,New York,New York County,40.73,-73.99,1,Coffee Shop,Japanese Restaurant,Cycle Studio,Indie Theater,Grocery Store,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Chinese Restaurant,Soba Restaurant,both
3,10004,New York,New York County,40.69,-74.02,1,Food Truck,Seafood Restaurant,Bike Rental / Bike Share,Theme Park Ride / Attraction,Pier,Baseball Field,Gift Shop,Park,Gym / Fitness Center,Beer Garden,both
4,10005,New York,New York County,40.71,-74.01,1,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Park,Pizza Place,Wine Shop,Gym / Fitness Center,Greek Restaurant,Café,both


### Visualizing the Clusters:

In [98]:
map_clusters = folium.Map(location=[39.828273, -98.579456], zoom_start=1)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(nyla_merged['latitude'], nyla_merged['longitude'], nyla_merged['Zip'], nyla_merged['Cluster']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Splitting the nyla_merged dataframe by cluster

### Cluster 1:

In [100]:
nyla_cluster1 = nyla_merged[nyla_merged['Cluster']==1]
nyla_cluster1.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
0,10001,New York,New York County,40.75,-74.00,1,Art Gallery,Coffee Shop,Italian Restaurant,Chinese Restaurant,Pizza Place,Health & Beauty Service,Theater,Pool,Cocktail Bar,Tapas Restaurant,both
2,10003,New York,New York County,40.73,-73.99,1,Coffee Shop,Japanese Restaurant,Cycle Studio,Indie Theater,Grocery Store,Italian Restaurant,Ice Cream Shop,Sushi Restaurant,Chinese Restaurant,Soba Restaurant,both
3,10004,New York,New York County,40.69,-74.02,1,Food Truck,Seafood Restaurant,Bike Rental / Bike Share,Theme Park Ride / Attraction,Pier,Baseball Field,Gift Shop,Park,Gym / Fitness Center,Beer Garden,both
4,10005,New York,New York County,40.71,-74.01,1,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Park,Pizza Place,Wine Shop,Gym / Fitness Center,Greek Restaurant,Café,both
5,10006,New York,New York County,40.71,-74.01,1,Coffee Shop,Hotel,Sandwich Place,Italian Restaurant,Park,Pizza Place,Wine Shop,Gym / Fitness Center,Greek Restaurant,Café,both


In [105]:
nyc_cluster1 = nyla_cluster1[(nyla_cluster1['county']=='New York County') | (nyla_cluster1['county']=='Bronx County') | (nyla_cluster1['county']=='Kings County') |
                      (nyla_cluster1['county']=='Queens County') | (nyla_cluster1['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 1.'.format(nyc_cluster1.shape[0]))

There are 225 New York City zipcodes in cluster 1.


In [106]:
la_cluster1 = nyla_cluster1[nyla_cluster1['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 1.'.format(la_cluster1.shape[0]))

There are 315 Los Angeles zipcodes in cluster 1.


### Cluster 2:

In [107]:
nyla_cluster2 = nyla_merged[nyla_merged['Cluster']==2]
nyla_cluster2.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
557,90745,Carson,Los Angeles County,33.82,-118.26,2,Building,Zoo,Filipino Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,both
561,90749,Carson,Los Angeles County,33.82,-118.26,2,Building,Zoo,Filipino Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,Fabric Shop,both


In [108]:
nyc_cluster2 = nyla_cluster2[(nyla_cluster2['county']=='New York County') | (nyla_cluster2['county']=='Bronx County') | (nyla_cluster2['county']=='Kings County') |
                      (nyla_cluster2['county']=='Queens County') | (nyla_cluster2['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 2.'.format(nyc_cluster2.shape[0]))

There are 0 New York City zipcodes in cluster 2.


In [109]:
la_cluster2 = nyla_cluster2[nyla_cluster2['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 2.'.format(la_cluster2.shape[0]))

There are 2 Los Angeles zipcodes in cluster 2.


### Cluster 3:

In [110]:
nyla_cluster3 = nyla_merged[nyla_merged['Cluster']==3]
nyla_cluster3.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
299,11405,Jamaica,Queens County,40.68,-73.78,3,Food,Deli / Bodega,Shopping Mall,Seafood Restaurant,Trail,Caribbean Restaurant,Pharmacy,Mexican Restaurant,Home Service,Discount Store,both
302,11413,Springfield Gardens,Queens County,40.66,-73.75,3,Deli / Bodega,Lake,Business Service,Home Service,Event Service,Event Space,Exhibit,Eye Doctor,Field,Factory,both
320,11431,Jamaica,Queens County,40.68,-73.78,3,Food,Deli / Bodega,Shopping Mall,Seafood Restaurant,Trail,Caribbean Restaurant,Pharmacy,Mexican Restaurant,Home Service,Discount Store,both
323,11434,Jamaica,Queens County,40.68,-73.78,3,Food,Deli / Bodega,Shopping Mall,Seafood Restaurant,Trail,Caribbean Restaurant,Pharmacy,Mexican Restaurant,Home Service,Discount Store,both
326,11439,Jamaica,Queens County,40.68,-73.78,3,Food,Deli / Bodega,Shopping Mall,Seafood Restaurant,Trail,Caribbean Restaurant,Pharmacy,Mexican Restaurant,Home Service,Discount Store,both


In [111]:
nyc_cluster3 = nyla_cluster3[(nyla_cluster3['county']=='New York County') | (nyla_cluster3['county']=='Bronx County') | (nyla_cluster3['county']=='Kings County') |
                      (nyla_cluster3['county']=='Queens County') | (nyla_cluster3['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 3.'.format(nyc_cluster3.shape[0]))

There are 6 New York City zipcodes in cluster 3.


In [112]:
la_cluster3 = nyla_cluster3[nyla_cluster3['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 3.'.format(la_cluster3.shape[0]))

There are 31 Los Angeles zipcodes in cluster 3.


### Cluster 4

In [113]:
nyla_cluster4 = nyla_merged[nyla_merged['Cluster']==4]
nyla_cluster4.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
1,10002,New York,New York County,40.71,-73.99,4,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink,both
7,10008,New York,New York County,40.71,-73.99,4,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink,both
40,10041,New York,New York County,40.71,-73.99,4,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink,both
41,10043,New York,New York County,40.71,-73.99,4,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink,both
43,10045,New York,New York County,40.71,-73.99,4,Café,Chinese Restaurant,Coffee Shop,Bridge,Latin American Restaurant,French Restaurant,Cantonese Restaurant,Other Great Outdoors,Mexican Restaurant,Skating Rink,both


In [114]:
nyc_cluster4 = nyla_cluster4[(nyla_cluster4['county']=='New York County') | (nyla_cluster4['county']=='Bronx County') | (nyla_cluster4['county']=='Kings County') |
                      (nyla_cluster4['county']=='Queens County') | (nyla_cluster4['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 4.'.format(nyc_cluster4.shape[0]))

There are 81 New York City zipcodes in cluster 4.


In [115]:
la_cluster4 = nyla_cluster4[nyla_cluster4['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 4.'.format(la_cluster4.shape[0]))

There are 6 Los Angeles zipcodes in cluster 4.


### Cluster 5:

In [116]:
nyla_cluster5 = nyla_merged[nyla_merged['Cluster']==5]
nyla_cluster5.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
160,10282,New York,New York County,40.72,-74.02,5,Park,Deli / Bodega,Rock Club,Dog Run,Soccer Field,Boat or Ferry,Volleyball Court,Health & Beauty Service,Seafood Restaurant,Mini Golf,both
164,10301,Staten Island,Richmond County,40.63,-74.09,5,Electronics Store,Bus Stop,Park,Gym,Dog Run,Deli / Bodega,Playground,Gym / Fitness Center,Flea Market,Food & Drink Shop,both
177,10314,Staten Island,Richmond County,40.60,-74.17,5,Playground,Convenience Store,Zoo,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,both
182,10455,Bronx,Bronx County,40.81,-73.91,5,Discount Store,Gym,Mexican Restaurant,Latin American Restaurant,Park,Pizza Place,Bakery,Italian Restaurant,Flight School,Falafel Restaurant,both
192,10465,Bronx,Bronx County,40.82,-73.83,5,Gym,Skate Park,Residential Building (Apartment / Condo),Park,Zoo,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,both


In [117]:
nyc_cluster5 = nyla_cluster5[(nyla_cluster5['county']=='New York County') | (nyla_cluster5['county']=='Bronx County') | (nyla_cluster5['county']=='Kings County') |
                      (nyla_cluster5['county']=='Queens County') | (nyla_cluster5['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 5.'.format(nyc_cluster5.shape[0]))

There are 24 New York City zipcodes in cluster 5.


In [118]:
la_cluster5 = nyla_cluster5[nyla_cluster5['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 5.'.format(la_cluster5.shape[0]))

There are 98 Los Angeles zipcodes in cluster 5.


### Cluster 6:

In [119]:
nyla_cluster6 = nyla_merged[nyla_merged['Cluster']==6]
nyla_cluster6.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
747,91606,North Hollywood,Los Angeles County,34.18,-118.39,6,Automotive Shop,Resort,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
752,91611,North Hollywood,Los Angeles County,34.18,-118.39,6,Automotive Shop,Resort,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
753,91612,North Hollywood,Los Angeles County,34.18,-118.39,6,Automotive Shop,Resort,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
756,91616,North Hollywood,Los Angeles County,34.18,-118.39,6,Automotive Shop,Resort,Fast Food Restaurant,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both


In [120]:
nyc_cluster6 = nyla_cluster6[(nyla_cluster6['county']=='New York County') | (nyla_cluster6['county']=='Bronx County') | (nyla_cluster6['county']=='Kings County') |
                      (nyla_cluster6['county']=='Queens County') | (nyla_cluster6['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 6.'.format(nyc_cluster6.shape[0]))

There are 0 New York City zipcodes in cluster 6.


In [121]:
la_cluster6 = nyla_cluster6[nyla_cluster6['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 6.'.format(la_cluster6.shape[0]))

There are 4 Los Angeles zipcodes in cluster 6.


### Cluster 7:

In [122]:
nyla_cluster7 = nyla_merged[nyla_merged['Cluster']==7]
nyla_cluster7.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
595,91007,Arcadia,Los Angeles County,34.12,-118.05,7,Gym / Fitness Center,Zoo,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,both
613,91077,Arcadia,Los Angeles County,34.12,-118.05,7,Gym / Fitness Center,Zoo,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,both


In [123]:
nyc_cluster7 = nyla_cluster7[(nyla_cluster7['county']=='New York County') | (nyla_cluster7['county']=='Bronx County') | (nyla_cluster7['county']=='Kings County') |
                      (nyla_cluster7['county']=='Queens County') | (nyla_cluster7['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 7.'.format(nyc_cluster7.shape[0]))

There are 0 New York City zipcodes in cluster 7.


In [124]:
la_cluster7 = nyla_cluster7[nyla_cluster7['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 7.'.format(la_cluster7.shape[0]))

There are 2 Los Angeles zipcodes in cluster 7.


### Cluster 8:

In [125]:
nyla_cluster8 = nyla_merged[nyla_merged['Cluster']==8]
nyla_cluster8.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
362,90027,Los Angeles,Los Angeles County,34.12,-118.28,8,Trail,Nightlife Spot,Park,Golf Course,Zoo,Farm,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,both
403,90068,Los Angeles,Los Angeles County,34.13,-118.33,8,Trail,Park,Zoo,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
437,90210,Beverly Hills,Los Angeles County,34.09,-118.41,8,Trail,Park,Zoo,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
470,90272,Pacific Palisades,Los Angeles County,34.07,-118.54,8,Trail,Zoo,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,both
471,90274,Palos Verdes Peninsula,Los Angeles County,33.77,-118.34,8,Trail,Baseball Field,Zoo,Electronics Store,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both


In [126]:
nyc_cluster8 = nyla_cluster8[(nyla_cluster8['county']=='New York County') | (nyla_cluster8['county']=='Bronx County') | (nyla_cluster8['county']=='Kings County') |
                      (nyla_cluster8['county']=='Queens County') | (nyla_cluster8['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 8.'.format(nyc_cluster8.shape[0]))

There are 0 New York City zipcodes in cluster 8.


In [127]:
la_cluster8 = nyla_cluster8[nyla_cluster8['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 8.'.format(la_cluster8.shape[0]))

There are 11 Los Angeles zipcodes in cluster 8.


### Cluster 9:

In [128]:
nyla_cluster9 = nyla_merged[nyla_merged['Cluster']==9]
nyla_cluster9.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
400,90065,Los Angeles,Los Angeles County,34.10,-118.22,9,Garden,Lawyer,Scenic Lookout,Business Service,Zoo,Factory,Farm,Falafel Restaurant,Fabric Shop,Fast Food Restaurant,both
631,91125,Pasadena,Los Angeles County,34.13,-118.12,9,Garden,Food,Tea Room,Zoo,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
632,91126,Pasadena,Los Angeles County,34.13,-118.12,9,Garden,Food,Tea Room,Zoo,Fast Food Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
636,91184,Pasadena,Los Angeles County,34.13,-118.16,9,Garden,Snack Place,Zoo,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
685,91350,Santa Clarita,Los Angeles County,34.43,-118.51,9,Garden,Zoo,Field,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,Eye Doctor,both


In [129]:
nyc_cluster9 = nyla_cluster9[(nyla_cluster9['county']=='New York County') | (nyla_cluster9['county']=='Bronx County') | (nyla_cluster9['county']=='Kings County') |
                      (nyla_cluster9['county']=='Queens County') | (nyla_cluster9['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 9.'.format(nyc_cluster9.shape[0]))

There are 0 New York City zipcodes in cluster 9.


In [130]:
la_cluster9 = nyla_cluster9[nyla_cluster9['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 9.'.format(la_cluster9.shape[0]))

There are 5 Los Angeles zipcodes in cluster 9.


### Cluster 0:

In [131]:
nyla_cluster0 = nyla_merged[nyla_merged['Cluster']==0]
nyla_cluster0.head()

,Zip,city,county,latitude,longitude,Cluster,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue,_merge
412,90077,Los Angeles,Los Angeles County,34.10,-118.45,0,Restaurant,Scenic Lookout,Café,Music Venue,Campground,Road,Farm,Falafel Restaurant,Factory,Fabric Shop,both
454,90245,El Segundo,Los Angeles County,33.91,-118.40,0,Restaurant,Golf Course,Zoo,Fast Food Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,Exhibit,both
537,90662,Pico Rivera,Los Angeles County,33.96,-118.09,0,BBQ Joint,Construction & Landscaping,Tree,Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,English Restaurant,Fast Food Restaurant,both
662,91307,West Hills,Los Angeles County,34.20,-118.69,0,Construction & Landscaping,River,Farm,Falafel Restaurant,Factory,Fabric Shop,Eye Doctor,Exhibit,Zoo,Fast Food Restaurant,both
708,91402,Panorama City,Los Angeles County,34.22,-118.44,0,Construction & Landscaping,Scenic Lookout,Zoo,Farmers Market,Empanada Restaurant,English Restaurant,Entertainment Service,Ethiopian Restaurant,Event Service,Event Space,both


In [132]:
nyc_cluster0 = nyla_cluster0[(nyla_cluster0['county']=='New York County') | (nyla_cluster0['county']=='Bronx County') | (nyla_cluster0['county']=='Kings County') |
                      (nyla_cluster0['county']=='Queens County') | (nyla_cluster0['county']=='Richmond County')]
print('There are {} New York City zipcodes in cluster 0.'.format(nyc_cluster0.shape[0]))

There are 0 New York City zipcodes in cluster 0.


In [133]:
la_cluster0 = nyla_cluster0[nyla_cluster0['county']=='Los Angeles County']
print('There are {} Los Angeles zipcodes in cluster 0.'.format(la_cluster0.shape[0]))

There are 7 Los Angeles zipcodes in cluster 0.
